In [1]:
__author__ = "Kexin Yu"
__version__ = "CS341, Stanford, Spring 2018"

In [2]:
import json
import os
import pandas as pd

In [3]:
news_data_home = 'news-coin-aggregations'
social_data_home = 'social-coin-aggregations'

coin_of_interest = 'BTC.jsonl'

def json_read_multiple_records(file):
    for line in open(file, mode="r"):
        yield json.loads(line)   
        
records = list(json_read_multiple_records(os.path.join(news_data_home, coin_of_interest)))

In [4]:
records[0]['views'].keys()

dict_keys(['country', 'key_phrases', 'concepts', 'count', 'categories', 'per_hour_sentiment', 'per_hour_reach'])

In [5]:
day0_sentiment_dict = records[0]['views']['per_hour_reach']['subViews']
day0_sentiment_dict.keys()

dict_keys(['1523919600000', '1523869200000', '1523894400000', '1523844000000', '1523923200000', '1523898000000', '1523872800000', '1523847600000', '1523851200000', '1523876400000', '1523901600000', '1523905200000', '1523880000000', '1523854800000', '1523908800000', '1523883600000', '1523858400000', '1523836800000', '1523912400000', '1523887200000', '1523862000000', '1523865600000', '1523890800000', '1523840400000', '1523916000000'])

In [6]:
import collections

def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# Per-hour sentiment

In [7]:
from datetime import datetime

sentiments = []

for record in records:
    daily_sentiment_dict = record['views']['per_hour_sentiment']['subViews']
    
    hour_ids = []
    frames = []

    for hour_id, d in daily_sentiment_dict.items():
        if d:
            hour_ids.append(hour_id)
            frames.append(flatten_dict(d))

    df = pd.DataFrame(frames)

    df['datetime'] = pd.Series(hour_ids)
    df['date'] = pd.Series([datetime.fromtimestamp(float(h)/1000).strftime('%Y-%m-%d') for h in hour_ids])
    df.set_index('datetime', inplace=True)
    df.drop(columns=['counts_type', 'mean_type'], inplace=True)
    
    sentiments.append(df)
    
df_sentiment = pd.concat(sentiments)
df_sentiment.drop(columns=['counts_counts_u'], inplace=True)
df_sentiment.sort_index(inplace=True)
df_sentiment.rename(columns={'counts_counts_n': 'news_neutral', 
                   'counts_counts_p': 'news_positive',
                   'counts_counts_v': 'news_negative',
                   'mean_statistics_MEAN': 'news_sentiment_mean'
                  }, inplace=True)
cols = ['date', 'news_neutral', 'news_positive', 'news_negative', 'news_sentiment_mean']
df_sentiment = df_sentiment[cols]
df_sentiment.fillna(0, inplace=True)
print(df_sentiment.head())
print(df_sentiment.tail())

                     date  news_neutral  news_positive  news_negative  \
datetime                                                                
1485997200000  2017-02-01           2.0            4.0            0.0   
1486004400000  2017-02-01           1.0            0.0            0.0   
1486008000000  2017-02-01           1.0            0.0            0.0   
1486011600000  2017-02-01           0.0            1.0            0.0   
1486015200000  2017-02-01           1.0            0.0            0.0   

               news_sentiment_mean  
datetime                            
1485997200000         6.666667e-01  
1486004400000         0.000000e+00  
1486008000000         9.694558e-07  
1486011600000         1.000000e+00  
1486015200000         1.652439e-08  
                     date  news_neutral  news_positive  news_negative  \
datetime                                                                
1523905200000  2018-04-16         384.0           11.0           24.0   
1523908800

# Per-hour document reach

In [8]:
records[0]['views']['per_hour_reach']['subViews']

reach = []

for record in records:
    daily_reach_dict = record['views']['per_hour_reach']['subViews']
    
    hour_ids = []
    frames = []

    for hour_id, d in daily_reach_dict.items():
        if d:
            hour_ids.append(hour_id)
            frames.append(flatten_dict(d))

    df = pd.DataFrame(frames)

    df['datetime'] = pd.Series(hour_ids)
    df['date'] = pd.Series([datetime.fromtimestamp(float(h)/1000).strftime('%Y-%m-%d') for h in hour_ids])
    df.set_index('datetime', inplace=True)
    df.drop(columns=['sum_type'], inplace=True)
    
    reach.append(df)
    
df_reach = pd.concat(reach)
df_reach.sort_index(inplace=True)
df_reach.rename(columns={'sum_statistics_SUM': 'news_reach_count'}, inplace=True)
cols = ['date', 'news_reach_count']
df_reach = df_reach[cols]
df_reach.fillna(0, inplace=True)
print(df_reach.head())
print(df_reach.tail())

                     date  news_reach_count
datetime                                   
1485997200000  2017-02-01          724308.0
1486004400000  2017-02-01           11832.0
1486008000000  2017-02-01           77971.0
1486011600000  2017-02-01           46797.0
1486015200000  2017-02-01             378.0
                     date  news_reach_count
datetime                                   
1523905200000  2018-04-16       254154812.0
1523908800000  2018-04-16       201147442.0
1523912400000  2018-04-16       327878870.0
1523916000000  2018-04-16       244126008.0
1523919600000  2018-04-16       150643500.0


In [9]:
df_sent_reach = df_sentiment.join(df_reach, lsuffix='', rsuffix='_right')
df_sent_reach.drop([col for col in df_sent_reach.columns if '_right' in col],axis=1,inplace=True)
df_sent_reach.tail()

,date,news_neutral,news_positive,news_negative,news_sentiment_mean,news_reach_count
datetime,,,,,,
1523905200000,2018-04-16,384.0,11.0,24.0,-0.030016,254154812.0
1523908800000,2018-04-16,233.0,152.0,27.0,0.302729,201147442.0
1523912400000,2018-04-16,598.0,38.0,13.0,0.038287,327878870.0
1523916000000,2018-04-16,512.0,11.0,5.0,0.011715,244126008.0
1523919600000,2018-04-16,296.0,8.0,20.0,-0.034630,150643500.0


In [10]:
df_sent_reach['news_daily_reach_count'] = df_sent_reach['news_reach_count'].groupby(df_sent_reach['date']).transform('sum')
df_sent_reach['news_daily_sentiment_mean'] = df_sent_reach['news_sentiment_mean'].groupby(df_sent_reach['date']).transform('mean')
df_sent_reach.tail(30)

,date,news_neutral,news_positive,news_negative,news_sentiment_mean,news_reach_count,news_daily_reach_count,news_daily_sentiment_mean
datetime,,,,,,,,
1523815200000,2018-04-15,263.0,6.0,5.0,0.004673,31482400.0,2.959193e+09,-0.024093
1523818800000,2018-04-15,189.0,8.0,23.0,-0.066782,186640773.0,2.959193e+09,-0.024093
1523822400000,2018-04-15,163.0,7.0,11.0,-0.023207,348485686.0,2.959193e+09,-0.024093
1523826000000,2018-04-15,178.0,6.0,9.0,-0.015032,64843407.0,2.959193e+09,-0.024093
1523829600000,2018-04-15,215.0,10.0,9.0,0.003687,44556315.0,2.959193e+09,-0.024093
1523833200000,2018-04-15,179.0,4.0,19.0,-0.072590,112344143.0,2.959193e+09,-0.024093
1523836800000,2018-04-15,174.0,7.0,18.0,-0.056712,66559376.0,2.959193e+09,-0.024093
1523840400000,2018-04-15,239.0,6.0,12.0,-0.021286,43522242.0,2.959193e+09,-0.024093
1523844000000,2018-04-15,169.0,52.0,7.0,0.199768,305179908.0,2.959193e+09,-0.024093


In [11]:
df_sent_reach.to_pickle('df_hourly_news_sentiment_reach.pickle')

# Daily Topics

In [12]:
from datetime import datetime

def get_bag_of_words(word_of_interest):
    """Collect all words from the given category and sort by count. 
    """
    lst = []
    timestamps = []

    for record in records:
        dict_ = record['views'][word_of_interest]['counts']
        sorted_dict = sorted(dict_.items(), key=lambda x: x[1], reverse=True)
        lst.append(sorted_dict)
        timestamps.append(sorted(list(record['views']['per_hour_sentiment']['counts'].keys()))[0])
    
    dates = [datetime.fromtimestamp(float(ts)/1000).strftime('%Y-%m-%d') for ts in timestamps]

    df = pd.DataFrame(
        {'date': dates,
         word_of_interest: lst,
        })

    df.set_index('date', inplace=True)
    df.sort_index(inplace=True)
                                
    return df

def phrase_to_words(df, word_of_interest):
    """Split phrase to words. 
    Example: ('Has Been', 17) -> ('Has', 17), ('Been', 17)
    """
    for index, row in df.iterrows():
        l1 = []
        for pair in row[word_of_interest]:
            l2 = []
            for word in pair[0].split():
                l2.append((word, pair[1]))
            l1.extend(l2)
        row[word_of_interest] = l1
    return df

In [13]:
df_concept = get_bag_of_words('concepts')
df_concept = phrase_to_words(df_concept, 'concepts')
print(df_concept.head())
print(df_concept.tail())

                                                     concepts
date                                                         
2017-02-01  [(Bitcoin, 48), (Has, 17), (Been, 17), (Canada...
2017-02-02  [(Bitcoin, 37), (United, 17), (States, 17), (J...
2017-02-03  [(Bitcoin, 15), (Cryptocurrency, 6), (For, 3),...
2017-02-04  [(Bitcoin, 18), (Cryptocurrency, 9), (Hacker, ...
2017-02-05  [(Bitcoin, 33), (Anonymity, 11), (United, 10),...
                                                     concepts
date                                                         
2018-04-11  [(Cryptocurrency, 11223), (Bitcoin, 9098), (Un...
2018-04-12  [(Cryptocurrency, 12276), (Bitcoin, 9985), (Tw...
2018-04-13  [(Cryptocurrency, 6498), (Bitcoin, 4296), (Uni...
2018-04-14  [(Cryptocurrency, 5758), (Bitcoin, 3738), (Uni...
2018-04-15  [(Cryptocurrency, 13378), (Bitcoin, 10470), (U...


In [14]:
df_keyword = get_bag_of_words('key_phrases')
df_keyword = phrase_to_words(df_keyword, 'key_phrases')
print(df_keyword.head())
print(df_keyword.tail())

                                                  key_phrases
date                                                         
2017-02-01  [(company, 13), (companies, 12), (digital, 12)...
2017-02-02  [(company, 15), (Bitcoin, 15), (investors, 9),...
2017-02-03  [(data, 5), (services, 5), (business, 3), (Pet...
2017-02-04  [(data, 3), (President, 2), (companies, 2), (c...
2017-02-05  [(company, 8), (state, 7), (users, 7), (collab...
                                                  key_phrases
date                                                         
2018-04-11  [(dollar, 2334), (Investors, 2287), (exchanges...
2018-04-12  [(dollar, 2613), (exchanges, 2487), (Investors...
2018-04-13  [(dollar, 2464), (Investors, 2365), (exchanges...
2018-04-14  [(dollar, 2360), (Investors, 2254), (exchanges...
2018-04-15  [(dollar, 2852), (Investors, 2822), (exchanges...


In [15]:
df_category = get_bag_of_words('categories')
print(df_category.head())
print(df_category.tail())

                                                   categories
date                                                         
2017-02-01  [(Business, 53), (Finance, 48), (Technology, 4...
2017-02-02  [(Business, 39), (Finance, 36), (Computers, 31...
2017-02-03  [(Business, 16), (Information Technology, 14),...
2017-02-04  [(Business, 19), (Finance, 18), (E-commerce, 1...
2017-02-05  [(Business, 33), (Finance, 33), (Information T...
                                                   categories
date                                                         
2018-04-11  [(Business, 9238), (Finance, 9044), (Informati...
2018-04-12  [(Business, 10136), (Finance, 9918), (Informat...
2018-04-13  [(Business, 4472), (Finance, 4162), (Informati...
2018-04-14  [(Business, 3809), (Finance, 3646), (Informati...
2018-04-15  [(Business, 10886), (Finance, 10488), (Technol...


In [16]:
df_country= get_bag_of_words('country')
print(df_country.head())
print(df_country.tail())

                                                      country
date                                                         
2017-02-01  [(us, 26), (au, 7), (in, 5), (gb, 5), (ng, 2),...
2017-02-02  [(us, 28), (gb, 3), (in, 2), (au, 1), (pt, 1),...
2017-02-03              [(us, 13), (ru, 1), (fr, 1), (ca, 1)]
2017-02-04  [(us, 15), (in, 1), (ph, 1), (ng, 1), (cn, 1),...
2017-02-05  [(us, 18), (ca, 3), (in, 2), (gb, 2), (be, 1),...
                                                      country
date                                                         
2018-04-11  [(us, 7540), (gb, 292), (ng, 247), (in, 191), ...
2018-04-12  [(us, 8463), (gb, 237), (ng, 205), (in, 180), ...
2018-04-13  [(us, 3660), (ng, 244), (in, 88), (gb, 68), (a...
2018-04-14  [(us, 3114), (ng, 111), (in, 80), (gb, 74), (s...
2018-04-15  [(us, 9522), (gb, 210), (ng, 186), (ca, 159), ...


In [17]:
df_topic = df_concept.join(df_keyword).join(df_category).join(df_country)
df_topic.head()

,concepts,key_phrases,categories,country
date,,,,
2017-02-01,"[(Bitcoin, 48), (Has, 17), (Been, 17), (Canada...","[(company, 13), (companies, 12), (digital, 12)...","[(Business, 53), (Finance, 48), (Technology, 4...","[(us, 26), (au, 7), (in, 5), (gb, 5), (ng, 2),..."
2017-02-02,"[(Bitcoin, 37), (United, 17), (States, 17), (J...","[(company, 15), (Bitcoin, 15), (investors, 9),...","[(Business, 39), (Finance, 36), (Computers, 31...","[(us, 28), (gb, 3), (in, 2), (au, 1), (pt, 1),..."
2017-02-03,"[(Bitcoin, 15), (Cryptocurrency, 6), (For, 3),...","[(data, 5), (services, 5), (business, 3), (Pet...","[(Business, 16), (Information Technology, 14),...","[(us, 13), (ru, 1), (fr, 1), (ca, 1)]"
2017-02-04,"[(Bitcoin, 18), (Cryptocurrency, 9), (Hacker, ...","[(data, 3), (President, 2), (companies, 2), (c...","[(Business, 19), (Finance, 18), (E-commerce, 1...","[(us, 15), (in, 1), (ph, 1), (ng, 1), (cn, 1),..."
2017-02-05,"[(Bitcoin, 33), (Anonymity, 11), (United, 10),...","[(company, 8), (state, 7), (users, 7), (collab...","[(Business, 33), (Finance, 33), (Information T...","[(us, 18), (ca, 3), (in, 2), (gb, 2), (be, 1),..."


In [18]:
df_topic.to_pickle('df_daily_news_topic.pickle')